# Fundamentals of Database Technologies - Assignment 3 (Part 1)

## Museums of London

### Question 1: Download as CSV File

The web data containing the lists of museums in London was imported into Google Docs using Google formula **ImportHTML** (the table containing the list is at index 2). Then the spreadsheet was downloaded as a CSV file using the 'Download as' feature in Google Sheets.  

### Question 2a: Number of Museums per Borough (Sorted Alphabetically)


In [1]:
import pandas as pd
import requests
import json
from time import sleep

museumsDF = pd.read_csv('MuseumsOfLondon.csv')

# Show first 5 rows only
museumsDF.head()

,Name,Image,District/ward,Borough,Region,Type,Summary
0,2 Willow Road,NaN,Hampstead,Camden,North,Historic house,"Operated by the National Trust, modernist terr..."
1,7 Hammersmith Terrace,NaN,Hammersmith,Hammersmith and Fulham,West,Historic house,"Former home of Emery Walker, friend and mentor..."
2,18 Stafford Terrace,NaN,Holland Park,Kensington and Chelsea,West,Historic house,"Also known as Linley Sambourne House, home of ..."
3,575 Wandsworth Road,NaN,Clapham,Lambeth,South,Historic house,"Operated by the National Trust, 18th century t..."
4,All Hallows-by-the-Tower Crypt Museum,NaN,Tower,City of London,North East,Religious,"Saxon-period Anglican church, museum of church..."


In [2]:
# a. What is the number of museums per Borough (sort alphabetically)?

pd.DataFrame(museumsDF.groupby('Borough').size().sort_index(), columns = ['Number of Museums'])

,Number of Museums
Borough,
Barking and Dagenham,2
Barnet,6
Bexley,4
Brent,1
Bromley,5
Camden,33
City of London,13
Croydon,2
Ealing,4


### Question 2b: Number of Historic Houses per Region (Sorted by Number of Museums)

In [3]:
# b. For Historic houses, what is their number per Region (sort from the most popular region)?
pd.DataFrame(museumsDF[museumsDF['Type'] == 'Historic house'].groupby('Region').size().sort_values(ascending = False), 
             columns = ['Number of Museums'])

,Number of Museums
Region,
North,14
West,11
South West,10
North East,7
South East,5
South,1


### Question 3: Geocoding

In [4]:
# 3. Try geocoding all museums (you may use name and perhaps something more, e.g. “London”). 
# Return a JSON (indent=2) with: Name, number of found locations, and if there is at least one: latitude and longitude.

def geoCode(address):
    retry = True
    jsonResults = []
    
    # Combine address parts
    finalAdd = ''
    for addPart in address:
        finalAdd = finalAdd + ', ' + addPart
    
    while (retry):
        r = requests.get("https://maps.googleapis.com/maps/api/geocode/json", params = {"address": finalAdd})
        jsonResults = r.json()
        if jsonResults['status'] == 'OK' or jsonResults['status'] == 'ZERO_RESULTS':
            retry = False
            #print(json.dumps(jsonResults, indent = 2))
        else:
            sleep(2)
            
    if jsonResults['status'] != 'ZERO_RESULTS':
        return {"Name": address[0], 
                "Locations": len(jsonResults['results']), 
                "Latitude": jsonResults['results'][0]['geometry']['location']['lat'], 
                "Longitude": jsonResults['results'][0]['geometry']['location']['lng']}
    else:
        return {"Name": address[0], 
                "Locations": len(jsonResults['results'])}
    

In [5]:
listdict = []

for i, r in museumsDF.iterrows():
    address = [r['Name']]
    if (pd.notnull(r['District/ward'])):
        address.append(r['District/ward'])
        
    address.append(r['Borough'])
    address.append('London')
        
    listdict.append(geoCode(address))
    sleep(2)

In [6]:
print(json.dumps(listdict, indent = 2))

[
  {
    "Locations": 0,
    "Name": "2 Willow Road"
  },
  {
    "Longitude": -0.2424588,
    "Latitude": 51.4898355,
    "Locations": 1,
    "Name": "7 Hammersmith Terrace"
  },
  {
    "Longitude": -0.1968857,
    "Latitude": 51.500476,
    "Locations": 1,
    "Name": "18 Stafford Terrace"
  },
  {
    "Longitude": -0.1406211,
    "Latitude": 51.4696864,
    "Locations": 1,
    "Name": "575 Wandsworth Road"
  },
  {
    "Locations": 0,
    "Name": "All Hallows-by-the-Tower Crypt Museum"
  },
  {
    "Locations": 0,
    "Name": "Anaesthesia Heritage Centre"
  },
  {
    "Longitude": -0.1516708,
    "Latitude": 51.5034719,
    "Locations": 1,
    "Name": "Apsley House"
  },
  {
    "Locations": 0,
    "Name": "Arsenal Football Club Museum"
  },
  {
    "Locations": 0,
    "Name": "Art in Perpetuity Trust"
  },
  {
    "Locations": 0,
    "Name": "artsdepot"
  },
  {
    "Locations": 0,
    "Name": "Baden-Powell House Exhibition"
  },
  {
    "Locations": 0,
    "Name": "Bank of Engla

In [7]:
latlongDF = pd.DataFrame(listdict, columns = ['Name', 'Locations', 'Latitude', 'Longitude'])

# Show first 5 rows only
latlongDF.head()

,Name,Locations,Latitude,Longitude
0,2 Willow Road,0,NaN,NaN
1,7 Hammersmith Terrace,1,51.489835,-0.242459
2,18 Stafford Terrace,1,51.500476,-0.196886
3,575 Wandsworth Road,1,51.469686,-0.140621
4,All Hallows-by-the-Tower Crypt Museum,0,NaN,NaN


### Question 4: Merge with Original DataFrame

In [8]:
# 4. Join (pd.join or pd.merge) it with the original DataFrame.

museumsLatLongDF = pd.merge(museumsDF, latlongDF, on = 'Name')

# Show first 5 rows only
museumsLatLongDF.head()

,Name,Image,District/ward,Borough,Region,Type,Summary,Locations,Latitude,Longitude
0,2 Willow Road,NaN,Hampstead,Camden,North,Historic house,"Operated by the National Trust, modernist terr...",0,NaN,NaN
1,7 Hammersmith Terrace,NaN,Hammersmith,Hammersmith and Fulham,West,Historic house,"Former home of Emery Walker, friend and mentor...",1,51.489835,-0.242459
2,18 Stafford Terrace,NaN,Holland Park,Kensington and Chelsea,West,Historic house,"Also known as Linley Sambourne House, home of ...",1,51.500476,-0.196886
3,575 Wandsworth Road,NaN,Clapham,Lambeth,South,Historic house,"Operated by the National Trust, 18th century t...",1,51.469686,-0.140621
4,All Hallows-by-the-Tower Crypt Museum,NaN,Tower,City of London,North East,Religious,"Saxon-period Anglican church, museum of church...",0,NaN,NaN


### Question 5: Average Latitude and Longitude for each Region

In [9]:
museumsLatLongDF.groupby('Region')['Latitude', 'Longitude'].mean()

,Latitude,Longitude
Region,,
North,51.535715,-0.137575
North East,51.528017,-0.003739
North West,NaN,NaN
South,51.469686,-0.140621
South East,51.453217,-0.032648
South West,51.452226,-0.281837
West,51.491971,-0.267579


In [10]:
# Number of museums without found location

museumsLatLongDF[museumsLatLongDF['Locations'] == 0].shape[0]

214